# Import packages

In [431]:
#conda install -c conda-forge sklearn-pandas
import pandas as pd 
import os
import datetime as dt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn_pandas import DataFrameMapper
from sklearn.cluster import KMeans
import hashlib as h
from collections import Counter
import math


## Interesting definition 

- k-anonymous : A table is k-anonymous if every record in the table is indistinguishable from at least k-1 other records, with respect to every set of quasi-identifiers
- l-diverse :  An equivalence class is l-diverse if it contains at least l distinct values for the sensitive attributes. A table is l-diverse if every equivalence class is l-diverse

# Directory choice

In [432]:
"""os.getcwd()
os.chdir('/Users/simonlegendre/Desktop/DATSMA2/AlgorithmsInDataScience/Projet1/Code/P3')

data = pd.read_csv("resources/dataset_HW3.csv")"""

data = pd.read_csv("../resources/dataset_HW3.csv")


In [433]:
data

print(data["gender"].describe())

#n/2000 = 0.487500

print(0.487500*2000)

count     2000
unique       2
top       male
freq      1025
Name: gender, dtype: object
975.0


# Visualazing data

In [434]:
#pd.reset_option('all')


"""data = pd.get_dummies(data, columns=["disease", "education", "employment", "marital_status","children", "ancestry", "accommodation"], 
                        prefix=["disease", "education", "employment", "marital_status","children", "ancestry", "accommodation"])

for col in data.columns:
    print(col)""" 

'data = pd.get_dummies(data, columns=["disease", "education", "employment", "marital_status","children", "ancestry", "accommodation"], \n                        prefix=["disease", "education", "employment", "marital_status","children", "ancestry", "accommodation"])\n\nfor col in data.columns:\n    print(col)'

#### For sociologists, to study the impact of stress and high-pressure environments on one’s health. They are looking for factors in people’s lives that could correlate to certain diseases or conditions. : 

They don't need to know any information about the identity of individuals. However, gender, dob and localisaiton  is really important information : 

- The id ( by name could be dropped (BE CARREFUL BECAUSE THIS INFORMATION COULD BE LINKED WITH THE SECOND PART OF THE EXERCICE)) 
- The gender could be replaced by 0 or 1 ( mais en vrai ca change pas grand chose)
- The dob : we don't need the day and the month of birth ==> Ne pourrait-on pas modifier grâce à une gaussienne de moyenne 0 sd 1 les dates de naissance ? On gagne en anonymisation, on ne perd pas grand chose 
- Zip code, we do not need the entire zipcode, we can keep only the first to digit.

==> Etant donnél'étude demandé, le risque n'est pas les identifiant mais les pseusos identifiants.

#### For the US department of Health and Human services chairman, to decide where to build new hospitals, and which departments (radiology, neurology, pulmonology, oncology...) to have in these, as the US federal government has just allocated money to build *5 new hospitals across the country* : 

A priori, je ne vois pas pourquoi on aurait besoin d'information sur les individus, ce qui nous importe est uniquement les maladies, la localicaiton, et pourquoi pas l'age des gens.

- Gender ne sert plus à rien
- l'année de naissance suffit 
- les deux premiers digit du zip code sont suffisant (étant donné que l'on va faire que 5 hopitauyx across the country)
- L'éducation on s'en branle
- Emploie on s'en fou
- Nombre d'enfant on s'en fou 
- Marital status on s'en branle
- ancestry on s'en fou 
- Nombre de véhicule ( est ce que ca peut être intéressant étant donné que c'est des hopitaux? ==> Ambulance tout ca)
- communte-time on s'en branle 
- accommodation on s'en fou 
- Maladie, pourrait on déjà regrouper les différentes maladies selon le service utilise à soigner les tiches ? On s'en branle de savoir si c'est un cancer du foie ou de la peau, ce qui est intéressant c'est qu'il faut un service d'oncologie



## Mofification de la base de données

### Change ID by hashing

In [435]:
d = {}

for id in data['id']:
    id_salted = str(id.replace(' ', '')+'arthurgeeraerd11751600')
    new_id = h.sha3_256(id_salted.encode()).hexdigest()
    d[id] = new_id

data_socio = data.replace(d)
d.clear()

# second dataset for the purpose of the 2nd use case (using salt : 'simonadrienkinartlegendre)

for id in data['id']:
    id_salted = str(id.replace(' ', '')+'simonadrienkinartlegendre')
    new_id = h.sha3_256(id_salted.encode()).hexdigest()
    d[id] = new_id

data_hospi = data.replace(d)
d.clear()

In [436]:
#pd.reset_option("all")

### Date of birth

In [437]:
# Download data 
data = pd.read_csv("../resources/dataset_HW3.csv")

#print(data["dob"])
new_dob = [0]*(len(data["dob"]))

# Projet hopitaux : on met les ages par dizaine d'année
for i in range(len(data["dob"])):

    # Modify DOB 
    s = 0 #np.random.normal(0, 2, 1)
    data.at[i,"dob"] = str(round(int(data["dob"][i][-4:]) +s, -1))[-2:]#[0]) 


# Projet sociologique : on met les ages avec une erreur suivant une normale.

"""for i in range(len(data["dob"])):

    # Modify DOB 
    s = np.random.normal(0, 2, 1)
    data.at[i,"dob"] = round(int(data["dob"][i][-4:]) +s[0]) 

    data.at[i,"zipcode"] = int((str(data["zipcode"][i])[:3]))"""




'for i in range(len(data["dob"])):\n\n    # Modify DOB \n    s = np.random.normal(0, 2, 1)\n    data.at[i,"dob"] = round(int(data["dob"][i][-4:]) +s[0]) \n\n    data.at[i,"zipcode"] = int((str(data["zipcode"][i])[:3]))'

### Variable catégorielles

Chaque fonction permet de gérer une variable catégorielle. On mets en entrée la valeur que l'on veut pour chaque catégorie possible, 
en retour, la database est modifiée avec les catégorie que l'on veut.

Exemple : 

change_education(data, "low", "low", "middle", "high", "high") va remplacer dans la dataBase Lessthan high school par low, high school par low, chachelor par moiddle, et master et phd par high

In [438]:
def change_education(data, less, high, bach, master, phd):

    mapping_education = {
        'Less than High School':less,
        'High School':high,
        'Bachelor':bach,
        'Masters':master,
        'PhD/md':phd,
    }

    data["education"] = data.education.map(mapping_education)


def change_gender(data, male, female):

    mapping_gender = {
        'male': male,
        'female': female
    }

    data["gender"] = data.gender.map(mapping_gender)


def change_employment(data, employed, retired, student, unemployed):
    mapping_employment = {
        'Employed' : employed,
        'Retired' : retired,
        'Student' : student,
        'Unemployed' : unemployed
    }

    data["employment"] = data.employment.map(mapping_employment)


def change_children(data, zero, un, deux, trois, quatre , cinq, six, sept, huit):
    mapping_children = {
        0 : zero,
        1 : un,
        2 : deux,
        3 : trois,
        4 : quatre,
        5 : cinq,
        6 : six,
        7 : sept,
        8 : huit
    }

    data["children"] = data.children.map(mapping_children)

def change_marital_status(data, divorced, married, single, widowed):
    mapping_marital_status = {
        "divorced" : divorced,
        "married" : married,
        "single" : single,
        "widowed" : widowed,

    }

    data["marital_status"] = data.marital_status.map(mapping_marital_status)


def change_ancestry(data, Africa, Asia, Europe, North_America, Oceania, South_America):
    mapping_ancestry = {
        "Africa" : Africa,
        "Asia" : Asia,
        "Europe" : Europe,
        "North America" : North_America,
        "Oceania" : Oceania,
        "South America" : South_America

    }

    data["ancestry"] = data.ancestry.map(mapping_ancestry)


def change_number_vehicles(data, zero, un, deux, trois):
    mapping_nmbr_vehicule = {
        0 : zero,
        1 : un,
        2 : deux,
        3 : trois,

    }

    data["number_vehicles"] = data.number_vehicles.map(mapping_nmbr_vehicule)


def change_accommodation(data, Own_flat, Own_house, Rent_flat, Rent_house, Rent_room):
    mapping_accommodation = {
        "Own flat" : Own_flat,
        "Own house" : Own_house,
        "Rent flat" : Rent_flat,
        "Rent house" : Rent_house,
        "Rent room" : Rent_room,
        
    }

    data["accommodation"] = data.accommodation.map(mapping_accommodation)

# for the sake of the first use case we gather all specific diseases under more general health department 

def change_disease(data, gastritis, heart_disease, hypertension, diabetes, skin_cancer, prostate_cancer, breast_cancer, schizophrenia, Alzheimer, multiple_sclerosis, endometriosis, kidney_disease, HIV_AIDS):
    mapping_disease = {
        'gastritis': gastritis ,
        'heart disease': heart_disease,
        "hypertension": hypertension,
        "diabetes": diabetes,
        "skin cancer": skin_cancer,
        "prostate cancer":  prostate_cancer,
        "breast cancer": breast_cancer, 
        "schizophrenia": schizophrenia,
        "Alzheimer's disease": Alzheimer, 
        "multiple sclerosis": multiple_sclerosis,
        "endometriosis": endometriosis, 
        "kidney disease": kidney_disease,
        "HIV/AIDS": HIV_AIDS 
    }

    data['disease'] = data.disease.map(mapping_disease)


In [439]:
# 'low' under highschool, 'high' elsewhere 
change_education(data, less= '0', high='0', bach='1', master='1', phd='1')

# '0' male, '1' female 
change_gender(data,male = 0, female = 1)

# active = {employed, student} unactive = {retired, unemployed}
change_employment(data, employed= '1', retired='0', student='1', unemployed='0')

# QUE FAIRE AVEC LES CHILDREN ?
change_children(data, "none", "more than one",'more than one','more than one','more than one','more than one','more than one','more than one','more than one')

# married = married, unmarried = others
change_marital_status(data, divorced='0', married='1', single='0', widowed='1')

# native : north america, foreigners : others BUT to much 'other' compare to native so might affect negatively l-diversity. Instead, we merge north/south america 
change_ancestry(data, Africa='Africa', Asia='Asia', Europe='Europe', North_America='America', Oceania='Oceania', South_America='America')

# 1 if got vehicule, 0 else 
change_number_vehicles(data, zero='none', un='one', deux='one', trois='one')

# owner = owner, renter = renter regardless of the type of the accomodaations (2/3 owner, maybe reset these proportion ? )
change_accommodation(data, Own_flat='1', Own_house='1', Rent_flat= '0', Rent_house='0', Rent_room='0')

# gathered under medical departments
change_disease(data, gastritis='gastro-enterolgy', heart_disease='cardiovascular', hypertension='cardiovascular', diabetes='endocrynology', 
    skin_cancer='oncology', prostate_cancer='oncology', breast_cancer='oncology', 
    schizophrenia='psychiatry', Alzheimer='neurology', multiple_sclerosis='neurology', endometriosis='gynecology', kidney_disease='nephrology', HIV_AIDS='interal mdeicine')

# 

In [440]:
data.head()

,id,gender,dob,zipcode,education,employment,children,marital_status,ancestry,number_vehicles,commute_time,accommodation,disease
0,Amelia Morrow,1,50,26904,0,0,more than one,1,Africa,one,0.0,1,neurology
1,Lillian Cole,1,40,78088,1,0,more than one,1,Africa,one,0.0,0,neurology
2,Lillian Simpson,1,70,51502,1,1,more than one,1,Asia,none,0.1,0,cardiovascular
3,Avery Richards,1,40,54080,1,0,more than one,1,Africa,none,0.0,0,gynecology
4,Sophia Alvarado,1,50,68785,0,0,more than one,1,Europe,none,0.0,0,gastro-enterolgy


import d'un base de données trouvée sur le net : http://structnet.com/instructions/zip_min_max_by_state.html

In [441]:
states = pd.read_csv("../resources/states.csv")
states.head()

,ID,State Name,ST,Zip Min,Zip Max
0,6,Alaska,AK,99501,99950
1,5,Alabama,AL,35004,36925
2,69,Arkansas,AR,71601,72959
3,8,Arkansas (Texarkana),AR,75502,75502
4,7,Arizona,AZ,85001,86556


In [442]:
dict_states = {}

for (i,state) in enumerate(states['State Name']):
    dict_states[state] = [states['Zip Min'].iloc[i,], states['Zip Max'].iloc[i,]]     

On créé le dictionnaire qui pour chaque clé : 'nom_etat', fait correspondre la valuer [zip_min, zip_max]

In [443]:
dict_zip_to_state = {}

for i,zip in enumerate(data['zipcode']):
    found = False
    for state, r in list(dict_states.items()):
        if zip in range(int(r[0]), int(r[1])):
            dict_zip_to_state[zip] = state
            found = True
    if not found:
        dict_zip_to_state[zip] = 'unknown state'


mapping ...

In [444]:
data['zipcode'] = data.zipcode.map(dict_zip_to_state)

In [445]:
#pd.reset_option('all')
data.head()

,id,gender,dob,zipcode,education,employment,children,marital_status,ancestry,number_vehicles,commute_time,accommodation,disease
0,Amelia Morrow,1,50,unknown state,0,0,more than one,1,Africa,one,0.0,1,neurology
1,Lillian Cole,1,40,Texas,1,0,more than one,1,Africa,one,0.0,0,neurology
2,Lillian Simpson,1,70,Iowa,1,1,more than one,1,Asia,none,0.1,0,cardiovascular
3,Avery Richards,1,40,Wisconsin,1,0,more than one,1,Africa,none,0.0,0,gynecology
4,Sophia Alvarado,1,50,Nebraska,0,0,more than one,1,Europe,none,0.0,0,gastro-enterolgy


### Variable continues 

!!! L'année de naissance minimum est de 1932 !!!

## k-Anonymity

### Test with sorting values

In [446]:
#pd.options.display.max_rows = None

In [447]:
data = data.sort_values(by=["gender", "dob", "education", "employment", "children", 
                "marital_status", "number_vehicles", "accommodation"])

data.head()


,id,gender,dob,zipcode,education,employment,children,marital_status,ancestry,number_vehicles,commute_time,accommodation,disease
520,Christian Nguyen,0,00,Arizona,0,0,none,0,Europe,none,0.0,0,nephrology
605,Jeremiah Soto,0,00,Iowa,0,0,none,0,Africa,none,0.0,0,interal mdeicine
723,Landon Nguyen,0,00,unknown state,0,0,none,0,Europe,none,0.0,0,gastro-enterolgy
434,Antonio Flores,0,00,Washington,0,0,none,0,Europe,one,0.0,0,oncology
1528,Logan Martin,0,00,West Virginia,0,0,none,0,America,one,0.0,0,interal mdeicine


In [448]:
#pd.set_option('display.max_rows', None)
data.head()

,id,gender,dob,zipcode,education,employment,children,marital_status,ancestry,number_vehicles,commute_time,accommodation,disease
520,Christian Nguyen,0,00,Arizona,0,0,none,0,Europe,none,0.0,0,nephrology
605,Jeremiah Soto,0,00,Iowa,0,0,none,0,Africa,none,0.0,0,interal mdeicine
723,Landon Nguyen,0,00,unknown state,0,0,none,0,Europe,none,0.0,0,gastro-enterolgy
434,Antonio Flores,0,00,Washington,0,0,none,0,Europe,one,0.0,0,oncology
1528,Logan Martin,0,00,West Virginia,0,0,none,0,America,one,0.0,0,interal mdeicine


In [451]:
def check_k_anonymity(data, k, list_variable) :
    group_by  = data[list_variable].groupby(list_variable)
    is_k_anonym = group_by[list_variable[0]].count() > k

    return is_k_anonym 


In [455]:
rep = check_k_anonymity(data, 2, ["gender", "dob", "education","number_vehicles", "accommodation"] )
print(rep.count() )

False


In [370]:
gr = data[["gender", "dob"]].groupby(["gender", "dob"])
is_k = (gr.count() > 2)
gr[["gender", "dob"][0]].count() > 2

gender  dob
0       00     True
        30     True
        40     True
        50     True
        60     True
        70     True
        80     True
        90     True
1       00     True
        30     True
        40     True
        50     True
        60     True
        70     True
        80     True
        90     True
Name: gender, dtype: bool

In [371]:
data

,id,gender,dob,zipcode,education,employment,children,marital_status,ancestry,number_vehicles,commute_time,accommodation,disease
520,Christian Nguyen,0,00,85116,0,0,none,0,Europe,none,0.00,0,nephrology
605,Jeremiah Soto,0,00,50959,0,0,none,0,Africa,none,0.00,0,interal mdeicine
723,Landon Nguyen,0,00,70553,0,0,none,0,Europe,none,0.00,0,gastro-enterolgy
434,Antonio Flores,0,00,98534,0,0,none,0,Europe,one,0.00,0,oncology
1528,Logan Martin,0,00,26608,0,0,none,0,America,one,0.00,0,interal mdeicine
...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,Valerie Pitts,1,90,55455,1,1,one,1,Africa,one,0.43,1,gastro-enterolgy
561,Elizabeth Jenkins,1,90,99952,1,1,one,1,Europe,one,1.77,1,oncology
927,Josephine Macias,1,90,69589,1,1,one,1,Africa,one,1.10,1,nephrology
1009,Josephine White,1,90,67974,1,1,one,1,America,one,2.10,1,gastro-enterolgy


### Test with k_means

In [209]:
data = pd.get_dummies(data, columns=["ancestry", "number_vehicles", "children"])
data_cluster = data 
data_cluster.drop(columns=["id", "disease"], inplace=True)



In [213]:
kmeans = KMeans(n_clusters=10, init='k-means++', random_state=0).fit(data_cluster)

c = Counter(kmeans.labels_)

print(c.items())

dict_items([(2, 178), (1, 211), (0, 208), (7, 199), (6, 179), (4, 203), (3, 207), (5, 215), (8, 203), (9, 197)])


En suivant la méthode de k means, lorsque l'on choisi 10 centres (choisi arbitrairement), nous obtenons des groupes équilibrés. un groupe n'est pas composé de peu d'observation. En d'autres termes, nous arrivons à créer via la méthode de k_means 10 groupes dont tous les individus sont proches sur l'ensemble des dimensions de quasi-identifier, ce qui signifie qiue dans chaque groupe, il y a environ 200 individus proches, et donc indistinguable l'un de l'autre.

## Modification supplémentaire

In [372]:
# We drop ID
data.drop(columns=["id"], inplace=True)

# Binariser certaines variables ( gender a déjà été fait)
"""data = pd.get_dummies(data, columns=["disease", "education", "employment", "marital_status", "ancestry", "accommodation"], 
                        prefix=["disease", "education", "employment", "marital_status", "ancestry", "accommodation"])"""

# Autre méthode
encoders = [(["gender"], LabelEncoder()), (["disease"], LabelEncoder()), (["education"], LabelEncoder()),(["employment"], LabelEncoder()), 
            (["marital_status"], LabelEncoder()),(["ancestry"], LabelEncoder()),(["accommodation"], LabelEncoder())]

mapper = DataFrameMapper(encoders, df_out=True)
new_cols = mapper.fit_transform(data.copy())

data = pd.concat([data.drop(columns=["gender","disease", "education", "employment", "marital_status", "ancestry", "accommodation"]),
        new_cols], axis="columns")


/Users/simonlegendre/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [373]:
data.nunique()

dob                   8
zipcode            1983
children              3
number_vehicles       3
commute_time         12
gender                2
disease               9
education             2
employment            2
marital_status        2
ancestry              5
accommodation         2
dtype: int64

In [374]:
# On regroupe les variables categorielles et continues entre elles

categorical = []
continuous = []

for c in list(data):
    col = data[c]
    nunique = col.nunique()
    if nunique < 20:
        categorical.append(c)
    else:
        continuous.append(c)


print(categorical)
print(continuous)

['dob', 'children', 'number_vehicles', 'commute_time', 'gender', 'disease', 'education', 'employment', 'marital_status', 'ancestry', 'accommodation']
['zipcode']
